# Projet 2 : Titanic
Pour ce deuxième projet, il s'agira de traiter une base de données beaucoup moins propre.  
En effet certaine données sont manquantes, et c'est cela qui rend ce projet utile, et intéressant.

On conserve le même environnement que pour le projet 1.

Le projet Titanic à pour but de prédire si un passager à survécu ou non.

## I. Importation des modules nécessaires  
On commence par importer les librairies et charger les données.

In [6]:
# Librairies de base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pour la modélisation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Affichage joli pour les graphiques
sns.set(style="whitegrid")
%matplotlib inline


In [7]:
# Charger le dataset train
train_df = pd.read_csv("data/train.csv")

# Charger le dataset test
test_df = pd.read_csv("data/test.csv")

# Vérifier les 5 premières lignes
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


On explore rapidement pour identifier les colonnes et les valeurs manquantes.

In [9]:

# Informations sur les types de colonnes et valeurs manquantes
train_df.info()

# Statistiques descriptives
train_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## II. Nettoyage et Feature Engineering
Comme le but de ce projet et de travailler sur des données manquantes, cette partie couvrera les domaines en question.

In [11]:
# Pour Age : moyenne ou médiane
imputer = SimpleImputer(strategy='median')
train_df['Age'] = imputer.fit_transform(train_df[['Age']])
test_df['Age'] = imputer.transform(test_df[['Age']])




imputer_emb = SimpleImputer(strategy='most_frequent')
train_df['Embarked'] = imputer_emb.fit_transform(train_df[['Embarked']])[:, 0]
test_df['Embarked'] = imputer_emb.transform(test_df[['Embarked']])[:, 0]


> Ici on initialise un 'imputer' : un outil qui sert à remplir les valeurs manquantes en fonction de la méthode qu'on lui a donné  
> - Pour l'âge, on utilise la méthode "median". Ainsi, toutes les valeurs manquantes de la colonne âge se verront attribuer la médiane de la colonne.
> - Pour une variable de type Object, on utilise aussi l'imputer mais avec la méthode "most frequent".  
> Ainsi les valeurs manquantes de la colonne Embarked seront remplacées par l'asset le plus cité dans la colonne.

>En ce qui concerne le "fit_transform" ou le "transform", on utilise le premier sur l'échantillon train, de sorte que le programme aille chercher la médiane sur le train (grâce au fit) pour l'implémenter sur les manquants du train (fit_transform) et sur les manquants du test (transform)

Ensuite, Il s'agit de réorganiser les données pour qu'elles soient plus faciles à manipuler.  
Premièrement, on extrait le titre du nom présent dans la colonne Name. On ne garde que le 'Mr' 'Mme' ou autres.  
Deuxièmement, on regroupe les autres titres 'rares' en une seule instance "Autres"  
Pour finir la première étape du feature engineering, on Encode le titre extrait, le sexe, et l'embarquation en données numériques.

In [15]:
# Extraire "Mr, Mme, etc.." à partir du Name
train_df['Title'] = train_df['Name'].str.extract(r"([A-Za-z]+)\.", expand=False)
test_df['Title'] = test_df['Name'].str.extract(r"([A-Za-z]+)\.", expand=False)

# Regrouper les titres rares en 'Other'
rare_titles = ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
train_df['Title'] = train_df['Title'].replace(rare_titles, 'Other')
test_df['Title'] = test_df['Title'].replace(rare_titles, 'Other')

# Encoder Title en numérique
title_mapping = {label: idx for idx, label in enumerate(train_df['Title'].unique())}
train_df['Title'] = train_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].map(title_mapping)

# Encoder le sexe
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

#Encoder EmBarked
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Embarked'], drop_first=True)


In [16]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

Pour finir, on supprime les colonnes que l'on considère inutiles.

In [18]:
drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']
train_df = train_df.drop(drop_cols, axis=1)
test_df = test_df.drop(drop_cols, axis=1)

In [19]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title,Embarked_Q,Embarked_S,FamilySize
0,0,3,0,22.0,1,0,7.2500,0,False,True,2
1,1,1,1,38.0,1,0,71.2833,1,False,False,2
2,1,3,1,26.0,0,0,7.9250,2,False,True,1
3,1,1,1,35.0,1,0,53.1000,1,False,True,2
4,0,3,0,35.0,0,0,8.0500,0,False,True,1


In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    int64  
 3   Age         891 non-null    float64
 4   SibSp       891 non-null    int64  
 5   Parch       891 non-null    int64  
 6   Fare        891 non-null    float64
 7   Title       891 non-null    int64  
 8   Embarked_Q  891 non-null    bool   
 9   Embarked_S  891 non-null    bool   
 10  FamilySize  891 non-null    int64  
dtypes: bool(2), float64(2), int64(7)
memory usage: 64.5 KB


## III. Séparation train/test et normalisation

In [22]:
# X = toutes les colonnes sauf Survived
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

> On sépare les colonnes de la colonne 'survived' qui est la colonne de test (c'est ce qu'on cherche à prédire)

In [89]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


> On utilise ici le train test split pour obtenir des variables différentes.  
> Le stratify sert à conserver les proportions de survie du dataset complet pour éviter les erreurs d'interprétations du modèle.

> **X_train** représente le jeu de donnée d'entrainement (80% du dataset)  
> **X_val** représente le jeu de donnée test sur lequel on va tester le modèle  
> **Y_train** représente la colonne des "survived" associé au X_train  
> **Y_val** représente les "survived" qui serviront à observer la prédiction du modèle. 

**N.B :** pour d'autres modèle que le randomForest, il est nécessaire de centrer réduire les variables,   
pour leur accorder le même poids dans les calculs  
Méthode :  

**scaler** = StandardScaler()

**X_train_scaled** = scaler.fit_transform(X_train)  
**X_val_scaled** = scaler.transform(X_val)


## IV. Étape 7 : Entraînement du modèle et évaluation

In [98]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [96]:
# Prédictions sur le set de validation
y_pred = model.predict(X_val)

# Score de précision
print("Accuracy:", accuracy_score(y_val, y_pred))

# Matrice de confusion
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# Rapport détaillé (précision, rappel, f1-score)
print("Classification Report:\n", classification_report(y_val, y_pred))


Accuracy: 0.8212290502793296
Confusion Matrix:
 [[95 15]
 [17 52]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.86      0.86       110
           1       0.78      0.75      0.76        69

    accuracy                           0.82       179
   macro avg       0.81      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [100]:
#dataset test
y_test_pred = model.predict(test_df)